In [18]:
import pandas as pd
from sqlalchemy import create_engine

In [19]:
# Path to SQLite DB file (this will create it in your project folder)
engine = create_engine('sqlite:///../data/vaccination.db')  # adjust path if needed

In [20]:
# Load cleaned CSVs
coverage = pd.read_csv("../Data/cleaned_coverage_data.csv")
incidence = pd.read_csv("../Data/cleaned_incidence_rate_data.csv")
reported = pd.read_csv("../Data/cleaned_reported_cases_data.csv")
vaccine_intro = pd.read_csv("../Data/cleaned_vaccine_introduction_data.csv")
schedule = pd.read_csv("../Data/cleaned_vaccine_schedule_data.csv")

In [21]:
# Save data into SQLite database
coverage.to_sql("coverage_data", con=engine, if_exists="replace", index=False)
incidence.to_sql("incidence_rate_data", con=engine, if_exists="replace", index=False)
reported.to_sql("reported_cases_data", con=engine, if_exists="replace", index=False)
vaccine_intro.to_sql("vaccine_introduction_data", con=engine, if_exists="replace", index=False)
schedule.to_sql("vaccine_schedule_data", con=engine, if_exists="replace", index=False)

print("✅ All tables loaded into vaccination.db successfully.")


✅ All tables loaded into vaccination.db successfully.


In [22]:
from sqlalchemy import create_engine, Column, Integer, Float, String, ForeignKey
from sqlalchemy.orm import declarative_base, relationship
from sqlalchemy.orm import sessionmaker

In [23]:
# Setup
Base = declarative_base()
engine = create_engine('sqlite:///../data/vaccination.db')  # Adjust path if needed

In [24]:
# TABLE: countries
class Country(Base):
    __tablename__ = 'countries'
    ISO_3_CODE = Column(String(3), primary_key=True)
    COUNTRY_NAME = Column(String(100))
    WHO_REGION = Column(String(10))

In [25]:
# TABLE: coverage_data
class CoverageData(Base):
    __tablename__ = 'coverage_data'
    CODE = Column(String(3), primary_key=True)
    YEAR = Column(Integer, primary_key=True)
    ANTIGEN = Column(String(50), primary_key=True)
    NAME = Column(String(100))
    ANTIGEN_DESCRIPTION = Column(String(255))
    COVERAGE_CATEGORY = Column(String(50))
    COVERAGE_CATEGORY_DESCRIPTION = Column(String(255))
    TARGET_NUMBER = Column(Integer)
    DOSES = Column(Integer)
    COVERAGE = Column(Float)

In [26]:
# TABLE: incidence_rate_data
class IncidenceRateData(Base):
    __tablename__ = 'incidence_rate_data'
    CODE = Column(String(3), primary_key=True)
    YEAR = Column(Integer, primary_key=True)
    DISEASE = Column(String(50), primary_key=True)
    NAME = Column(String(100))
    DISEASE_DESCRIPTION = Column(String(255))
    DENOMINATOR = Column(String(100))
    INCIDENCE_RATE = Column(Float)

In [27]:
# TABLE: reported_cases_data
class ReportedCasesData(Base):
    __tablename__ = 'reported_cases_data'
    CODE = Column(String(3), primary_key=True)
    YEAR = Column(Integer, primary_key=True)
    DISEASE = Column(String(50), primary_key=True)
    NAME = Column(String(100))
    DISEASE_DESCRIPTION = Column(String(255))
    CASES = Column(Integer)

In [28]:
# TABLE: vaccine_introduction_data
class VaccineIntroductionData(Base):
    __tablename__ = 'vaccine_introduction_data'
    ISO_3_CODE = Column(String(3), ForeignKey('countries.ISO_3_CODE'), primary_key=True)
    DESCRIPTION = Column(String(100), primary_key=True)
    COUNTRY_NAME = Column(String(100))
    WHO_REGION = Column(String(10))
    YEAR = Column(Integer)
    INTRO = Column(Integer)

In [29]:
# TABLE: vaccine_schedule_data
class VaccineScheduleData(Base):
    __tablename__ = 'vaccine_schedule_data'
    ISO_3_CODE = Column(String(3), ForeignKey('countries.ISO_3_CODE'), primary_key=True)
    YEAR = Column(Integer, primary_key=True)
    VACCINECODE = Column(String(50), primary_key=True)
    SCHEDULEROUNDS = Column(Integer, primary_key=True)
    COUNTRY_NAME = Column(String(100))
    WHO_REGION = Column(String(10))
    VACCINE_DESCRIPTION = Column(String(255))
    TARGETPOP = Column(String(50))
    TARGETPOP_DESCRIPTION = Column(String(255))
    GEOAREA = Column(String(50))
    AGEADMINISTERED = Column(String(50))
    SOURCECOMMENT = Column(String)

In [30]:
# Create all tables
Base.metadata.create_all(engine)

print("✅ SQLite tables with PKs and FKs created successfully.")

✅ SQLite tables with PKs and FKs created successfully.


In [31]:

# ✅ Create 'countries' table with ISO_3_CODE, COUNTRY_NAME, ISO_2_CODE, WHO_REGION
import pandas as pd
import sqlite3
import pycountry

# Load coverage data
coverage_df = pd.read_csv("../data/cleaned_coverage_data.csv")  # Ensure this file is in the same directory

# Extract and rename
country_df = coverage_df[['CODE', 'NAME', 'GROUP_NAME']].drop_duplicates()
country_df.columns = ['ISO_3_CODE', 'COUNTRY_NAME', 'WHO_REGION']

# Map ISO_3_CODE to ISO_2_CODE
def get_iso2_from_iso3(iso3):
    try:
        return pycountry.countries.get(alpha_3=iso3).alpha_2
    except:
        return None

country_df['ISO_2_CODE'] = country_df['ISO_3_CODE'].apply(get_iso2_from_iso3)

# Drop rows with missing ISO_2_CODE
country_df = country_df.dropna(subset=['ISO_2_CODE'])

# Reorder columns
country_df = country_df[['ISO_3_CODE', 'COUNTRY_NAME', 'ISO_2_CODE', 'WHO_REGION']]

# Save to SQLite
conn = sqlite3.connect("vaccination.db")  # Adjust path if needed
country_df.to_sql("countries", conn, if_exists="replace", index=False)

# Confirm
cursor = conn.cursor()
cursor.execute("SELECT COUNT(*) FROM countries")
print(f"✅ Inserted {cursor.fetchone()[0]} countries into 'countries' table.")
conn.close()


✅ Inserted 213 countries into 'countries' table.
